In [ ]:
import os
import boto3
import sagemaker
import torch
from sagemaker.pytorch import PyTorch

from sagemaker import get_execution_role
role = get_execution_role()
account_id = role.split(':')[4]
sess = sagemaker.Session()
region = boto3.session.Session().region_name
print("Region = {}".format(region))
    
sm = boto3.Session().client('sagemaker')

In [ ]:
!sudo apt-get install -y libsndfile1

In [ ]:
modelfolder = "tyhac-fastai-2021-08-22-10-37-01-521"
rawbucket = sess.default_bucket()
model_artefact = f's3://{rawbucket}/sagemaker-covid-tyhac-fastai/models/{modelfolder}/output/model.tar.gz'
print(model_artefact)

In [ ]:
# optionally download the model archive locally
# !mkdir {modelfolder}
filename = modelfolder + '/' + 'model.tar.gz'
!aws s3 cp {model_artefact} {filename}
!tar -zxvf {filename} -C {modelfolder} 'export.pkl' 'roc_curve.jpg' 'confusion_matrix.jpg' 'dataset_test.csv'

In [ ]:
# %%writefile inference/serve.py
import logging, requests, os, io, glob, time, pathlib
from fastai.vision.all import *
import boto3
s3 = boto3.client('s3')

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

JSON_CONTENT_TYPE = 'application/json'
# JPEG_CONTENT_TYPE = 'image/jpeg'

# loads the model into memory from disk and returns it
def model_fn(model_dir):
    logger.info('model_fn')
    path = Path(model_dir)
    learn = load_learner(model_dir + '/export.pkl')
    logger.info('TYHAC: Model has been loaded')
    return learn

def input_fn(request_body, request_content_type):
    logger.info('TYHAC: Deserializing the input data.')
    
    if request_content_type == 'text/csv':
        tmp=request_body
        print('tyhac body: ', tmp)
        bucket=tmp[:tmp.index('/')]
        print("bucket: {}".format(bucket))
        obj=tmp[tmp.index('/')+1:]
        print("object: {}".format(obj))
        file = s3.download_file(bucket, obj, 'audioinput.wav')
        print("audio input file size: {}".format(os.path.getsize('audioinput.wav')))
        return 'audioinput.wav'
        # waveform, sample_rate = torchaudio.load('/audioinput.wav')
    
    # process an image uploaded to the endpoint
    # if content_type == JPEG_CONTENT_TYPE: return open_image(io.BytesIO(request_body))
    # process a URL submitted to the endpoint
    # if content_type == JSON_CONTENT_TYPE:
    #    img_request = requests.get(request_body['url'], stream=True)
    #    return open_image(io.BytesIO(img_request.content))
    raise Exception('Requested unsupported ContentType in content_type: {}'.format(request_content_type))

# Perform prediction on the deserialized object, with the loaded model
def predict_fn(input_object, model):
    logger.info("TYHAC: Calling model for predict")
    start_time = time.time()
    predict_class,predict_idx,predict_values = model.predict(input_object)
    print("--- Inference time: %s seconds ---" % (time.time() - start_time))
    print(f'Predicted class is {str(predict_class)}')
    print(f'Predict confidence score is {predict_values[predict_idx.item()].item()}')
    return dict(class_name = str(predict_class),
        confidence = predict_values[predict_idx.item()].item())

# Serialize the prediction result into the desired response content type
def output_fn(prediction, accept=JSON_CONTENT_TYPE):        
    logger.info('TYHAC: Serializing the predicted output.')
    if accept == JSON_CONTENT_TYPE: return json.dumps(prediction), accept
    raise Exception('Requested unsupported ContentType in Accept: {}'.format(accept)) 

if __name__ == "__main__":
    model = model_fn('tyhac-fastai-2021-08-22-23-10-40-004')

    print(model.dls.vocab)

    audio = input_fn('sagemaker-ap-southeast-2-ACCOUNTID/sagemaker-covid-tyhac-fastai/data/audio/0sIeyohqXMOGTqPA7RiiuyJ3AOt2-cough-shallow.wav', 'text/csv')
    
    predict_fn(audio, model)

In [ ]:
# from sagemaker.predictor import Predictor

# class ImagePredictor(Predictor):
#    def __init__(self, endpoint_name, sagemaker_session):
#        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=None, 
#                         deserializer=None, content_type='test/csv')

In [ ]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(model_data=model_artefact,
                   source_dir='inference/',
                   role=role, framework_version='1.5.1', py_version='py3', entry_point='serve.py',
                   image_uri='ACCOUNTID.dkr.ecr.ap-southeast-2.amazonaws.com/tyhac-sagemaker-inference-fastai:1.0-cpu-py36'
                  )

predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium', wait=False) # ml.t2.medium
print("Inference endpoint name: {}".format(model.endpoint_name))

In [ ]:
# url = <some url of an image to test>
# img_bytes = requests.get(url).content
response = predictor.predict('sagemaker-ap-southeast-2-ACCOUNTID/sagemaker-covid-tyhac-fastai/data/audio/0sIeyohqXMOGTqPA7RiiuyJ3AOt2-cough-shallow.wav');
print(response)

In [ ]:

## make a prediction
import boto3

client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName=model.endpoint_name,
    Body='sagemaker-ap-southeast-2-ACCOUNTID/sagemaker-covid-tyhac-fastai/data/audio/0sIeyohqXMOGTqPA7RiiuyJ3AOt2-cough-shallow.wav'.format(account_id),
    ContentType='text/csv',
)

print("The probability of positive label is {}".format(response['Body'].read().decode("utf-8")))

In [ ]:
predictor.delete_model()
predictor.delete_predictor()